In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import time
import pickle

In [45]:
class Pipeline:
    
    def __init__(self):
        
        #dataset directory
        self.ROOT = pathlib.Path.cwd()/'PACKAGE'
        self.DATASET_DIR = self.ROOT/'DATASET'
        self.MODEL_DIR = self.ROOT/'MODEL'
        
        #data
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        
        #parameters
        self.class_mapper = class_mapper = {0:'T-shirt', 1:'Trouser', 2:'Pullover', 3:'Dress', 4:'Coat', 5:'Sandal', 6:'Shirt',
               7:'Sneaker', 8:'Bag', 9:'Ankle boot'}
        
        self.input_shape = None
        
        #model
        self.model = None
        
    def create_directory(self):
        start = time.time()
        template = [self.ROOT, self.DATASET_DIR, self.MODEL_DIR]
        for directory in template:
            if directory.exists() == True:
                continue
            else:
                os.mkdir(directory)
        stop = time.time()
        total_time = stop - start
        print('Directory created, it took {0} s'.format(round(total_time, ndigits = 2)))
        del template
        
        

    def data_load(self):

        start = time.time()
        (self.X_train, self.y_train),(self.X_test, self.y_test) = tf.keras.datasets.fashion_mnist.load_data()
        stop = time.time()
        total_time = stop - start
        print('Data loaded, it took {0} s'.format(round(total_time, ndigits = 2)))
    
    def data_preprocess_and_save(self):
        start = time.time()
        
        self.X_train = self.X_train/255
        self.X_test = self.X_test/255
        self.y_train = tf.keras.utils.to_categorical(self.y_train)
        self.y_test = tf.keras.utils.to_categorical(self.y_test)
        
        with open(self.DATASET_DIR/'X_train.pkl', 'wb') as file:
            pickle.dump(self.X_train, file)
        with open(self.DATASET_DIR/'X_test.pkl', 'wb') as file:
            pickle.dump(self.X_test, file)
        with open(self.DATASET_DIR/'y_train.pkl', 'wb') as file:
            pickle.dump(self.y_train, file)
        with open(self.DATASET_DIR/'y_test.pkl', 'wb') as file:
            pickle.dump(self.y_test, file)
            
        stop = time.time()
        total_time = stop - start
        
        print('Data prepared and saved in directory, it took {0} s'.format(round(total_time, ndigits = 2)))
        
    def show_data(self, to_show = 16):
        fig = plt.figure(figsize = (12,12))
        
        for i in range(1, to_show+1):
            random_number = np.random.randint(low = 0, high = len(self.X_train))
            ax = fig.add_subplot(to_show/4, 4, i)
            ax.imshow(self.X_train[random_number])
            ax.set_title(self.class_mapper[self.y_train[random_number].argmax()], 
                         color = 'white', fontsize = 14, fontweight = 'bold')
            ax.axis('off')  
        plt.subplots_adjust(hspace = 0.4, wspace = 0)
        plt.show()
        
    def model_build(self):
        self.input_shape = tf.expand_dims(self.X_train[0], axis = 2).shape
        
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Conv2D(filters = 16, kernel_size  =(2,2), padding = 'same',
                                              strides = 1, input_shape = self.input_shape))
        self.model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2), strides = 2, padding = 'same'))
        self.model.add(tf.keras.layers.BatchNormalization())
        self.model.add(tf.keras.layers.Activation('relu'))
        
        self.model.add(tf.keras.layers.Conv2D(filters = 16, kernel_size  =(2,2), padding = 'same',
                                              strides = 1))
        self.model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2), strides = 2, padding = 'same'))
        self.model.add(tf.keras.layers.BatchNormalization())
        self.model.add(tf.keras.layers.Activation('relu'))
                       
        self.model.add(tf.keras.layers.Flatten())
        
        self.model.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
        self.model.add(tf.keras.layers.Dense(units = 10, activation = 'sigmoid'))
    
        self.model.summary()
    
    
    def train_model(self):
        
    def train_pipeline(self):
        self.create_directory()
        self.data_load()
        self.data_preprocess_and_save()
        #self.show_data()
        self.model_build()
        
        
        

In [46]:
obj1 = Pipeline()
obj1.train_pipeline()

Directory created, it took 0.0 s
Data loaded, it took 1.38 s
Data prepared and saved in directory, it took 4.94 s
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 16)        80        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 16)        64        
_________________________________________________________________
activation_4 (Activation)    (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 16)        1040      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 16)          0         
______